# Build a Dashboard Application with Plotly Dash

In [1]:
!pip install dash

     |████████████████████████████████| 7.5 MB 4.7 MB/s eta 0:00:01


In [14]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
# import dash_html_components as html
# import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [15]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)
spacex_df.head(3)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0


In [16]:
launch_Site_List = list(spacex_df['Launch Site'].unique())
[_ for _ in launch_Site_List]

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [17]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [11]:
custom_colors = {0:'red', 1:'blue'}

In [21]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[{'label':'All Sites', 'value': 'ALL'},
                                                     *[{'label': _, 'value': _} for _ in launch_Site_List], # dropdown list using for loop
                                                    ],
                                            # value='site1', # placeholder ထည့်လျှင် default value ထည့်စရာမလို
                                            placeholder='Select one option',
                                            searchable=True,
                                            ),
                                html.Br(),


                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"), # use for paragraph
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=min_payload,
                                                max=max_payload,
                                                step=1000,
                                                marks={i: str(i) for i in range(int(min_payload), int(max_payload) + 1, 1000)},
                                                value = [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_d = spacex_df
        fig = px.pie(filtered_d, names='Launch Site',
                     title='Total Successful Launches by Site') 
        
    else:
        filtered_d = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_d, names='class',
             title=f'Total Successful Launches for Site: {entered_site}') 
        
    return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))

def update_scatter_chart(entered_site, payload_range):
    
    filtered_df = spacex_df
    
    # Filter by selected site
    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
    
    # Filter by selected payload range
    min_payload, max_payload = payload_range
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= min_payload) &
                              (filtered_df['Payload Mass (kg)'] <= max_payload)]
    
    # Create scatter plot
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                     color='class',
                     color_discrete_map=custom_colors,
                     title=f'Success vs. Failure by Payload Mass for Site: {entered_site}' if entered_site != 'ALL' else 'Success vs. Failure by Payload Mass')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=8080, jupyter_mode='external')

Dash app running on http://127.0.0.1:8080/
